<a href="https://colab.research.google.com/github/r-dube/CICIDS/blob/main/fj_ltsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the modules used
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant 
from keras.optimizers import Adam
from keras import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [2]:
# NLTK to remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# For reproducible results
import random as rn
import os
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [4]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [5]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [6]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (3576, 18)
Not fraudulent 3395 Fraudulent 181
             job_id  telecommuting  ...  has_questions   fraudulent
count   3576.000000    3576.000000  ...    3576.000000  3576.000000
mean    8882.569911       0.043624  ...       0.493568     0.050615
std     5137.458280       0.204286  ...       0.500029     0.219241
min        1.000000       0.000000  ...       0.000000     0.000000
25%     4446.500000       0.000000  ...       0.000000     0.000000
50%     8862.500000       0.000000  ...       0.000000     0.000000
75%    13286.750000       0.000000  ...       1.000000     0.000000
max    17879.000000       1.000000  ...       1.000000     1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                    0
title                     0
location                 70
department             2324
salary_range           2988
company_profile         692
description               0
requirements            549
benefits               1447
telecommuting            

In [7]:
# Utilities to clean text

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [10]:
# clean text
df['text'] = df['text'].map(lambda x: remove_URL(x))
df['text'] = df['text'].map(lambda x: remove_html(x))
df['text'] = df['text'].map(lambda x: remove_emoji(x))
df['text'] = df['text'].map(lambda x: remove_punct(x))
df['text'] = df["text"].map(remove_stopwords)

In [11]:
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

text = df['text']

counter = counter_word(text)
num_words = len(counter)
print ("Number of words: ", num_words)

# Max number of words in a sequence
max_length = 250

Number of words:  64265


In [12]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

In [13]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (3039, 250)
Shape of test (537, 250)


In [14]:
# median_length = sorted([len(x) for x in train_sequences])[len(train_sequences) // 2]
# print (median_length)
# Sanity check for input
# print(train_text[0])
# print(train_sequences[0])
# print(train_padded[0])

In [15]:
model = Sequential()

hidden_size = 32
model.add(Embedding(num_words, hidden_size*2, input_length=max_length))
model.add(Bidirectional(LSTM(hidden_size, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.01)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', metrics.FalsePositives(), metrics.FalseNegatives()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 64)           4112960   
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 64)           24832     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 4,137,857
Trainable params: 4,137,857
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_padded, train_labels, epochs=5)

Epoch 1/5
95/95 [==============================] - 48s 503ms/step - loss: 0.2006 - accuracy: 0.9411 - false_positives: 29.0000 - false_negatives: 150.0000
Epoch 2/5
95/95 [==============================] - 49s 519ms/step - loss: 0.0579 - accuracy: 0.9812 - false_positives: 9.0000 - false_negatives: 48.0000
Epoch 3/5
95/95 [==============================] - 50s 530ms/step - loss: 0.0094 - accuracy: 0.9974 - false_positives: 2.0000 - false_negatives: 6.0000
Epoch 4/5
95/95 [==============================] - 49s 518ms/step - loss: 0.0011 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 5/5
95/95 [==============================] - 48s 510ms/step - loss: 5.2571e-04 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00


In [17]:
pred = model.predict(test_padded)
pred = np.around(pred, decimals = 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)
print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))

Accuracy score: 0.9907 F1 score: 0.8837
